In [1]:
import os
import weaviate
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
client = weaviate.Client(
    url=os.environ["WEAVIATE_URL"],
    auth_client_secret=weaviate.AuthApiKey(api_key=os.environ['WEAVIATE_APIKEY']),
    additional_headers={
        "X-OpenAI-Api-Key": os.environ['OPENAI_APIKEY']
    }
    )

In [ ]:
# Test connection
print(client.is_live())
print(client.is_ready())

#### Set you Hasura Console

Execute the following steps on your Hasura console

1. Go to postgres table and track candidate and application
2. Add Weaviate Agent by  clicking `Manage` on Data tab under Database panel on left
    - Click `Data connector Agents` 
    - Click `Add Agent`
    - Name => weaviate
    - URL => weaviate-gdc:8100
3. Connect Weaviate db 
    - Click `Connect Database`
    - Select weaviate from db selection
    - Connect Existing Database
    - Type Database name
    - Pick apiKey and host from weaviate cluster details
    


1. Set relationship
    - From weaviate table -> Relationship
    - Give name `application_relationship`
    - From resume (application_id) to application (id)
2. Set permission
    - Go to Resume
    - Create new role "manager" and update permissions for "select"
        - {
            "application_relationship": {
                "hiring_manager": {
                "_eq": "x-hasura-manager-id"
                }
            }
            }
    - Select permission for columns (Toggle all)
3. Fetch by permission
    - Go to API
    - 

In [3]:
!pip install gql

  Using cached gql-3.4.1-py2.py3-none-any.whl (65 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 3.3 MB/s eta 0:00:00a 0:00:01


In [4]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport


transport = AIOHTTPTransport(url="http://localhost:8080/v1/graphql")
client = Client(transport=transport, fetch_schema_from_transport=True)

AssertionError: Cannot run client.execute(query) if an asyncio loop is running. Use 'await client.execute_async(query)' instead.

#### Usecases

In [41]:
import os
from pprint import pprint

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


from dotenv import load_dotenv
load_dotenv()

llm = OpenAI(model="text-davinci-003",
             openai_api_key=os.environ['OPENAI_APIKEY'])

chatllm = ChatOpenAI(model="gpt-3.5-turbo",
                     openai_api_key=os.environ['OPENAI_APIKEY'])

#### Usecase 1 - Question resume

In [9]:
result = (
    client.query.get(
    "Resume", 
    [
        "application_id", 
        "content"
    ]
    )
    .with_limit(1)
    .do()
)
print(result)

{'data': {'Get': {'Resume': [{'application_id': 378, 'content': "         TEACHER       Summary     Dedicated teaching and counseling educator with 17 of experience.  I have successfully taught English to students, both children and adults, \xa0that were new to the country as well as helped and supported them as they \xa0assimilated to the US school system and culture.\xa0I have also successfully implemented an IB and a regular Spanish curriculum to Middle and High School students with different levels of language proficiency.  During my career, I have implemented \xa0the Alweus Bullying Prevention Program, served in LPAC Committees and Student Assistance Programs, administered TELPAS Online Assessments and collaborated with other educators as a TELPAS trainer.\xa0       Core Qualifications          MA Education  Counseling Certification  Counseling Internship in Elementary School and juvenile correctional facility      High School Spanish endorsement  Bilingual Endorsement\xa0  IB cur

In [45]:
resume_question_prompt_template = PromptTemplate.from_template(
    """
    You are a helpful Question Answering bot. You are provided with Resume content and a question on the content.
    Answer the question from the content.
    If unsure, answer with "I don't know".

    Question: {question}
    Resume: {resume_content}

    """
)


chain = LLMChain(llm=llm, prompt=resume_question_prompt_template)


In [46]:
print(chain.run({
    "question": "How many years of experience does the candidate have?",
    "resume_content": result['data']["Get"]['Resume'][0]['content']
}))


Answer: The candidate has 17 years of experience.


In [48]:
pprint(chain.run({
    "question": "Summarise the skills of the candidate in 2-3 lines.",
    "resume_content": result['data']["Get"]['Resume'][0]['content']
}))

('\n'
 'Answer: The candidate is a dedicated teaching and counseling educator with '
 '17 years of experience. They have successfully taught English and Spanish to '
 'students of all ages, implemented an IB curriculum, administered TELPAS '
 'Online Assessments, and collaborated as a TELPAS trainer. They hold a MA in '
 'Education, Counseling Certification, and Bilingual Endorsement, and have '
 'skills such as counseling, leadership, workshops, goal setting, and working '
 'with parents.')


#### Use case -2 Find most relevant 